# Function Generation for the Training of λ-Nets

## Specitication of Experiment Settings

In [1]:
#######################################################################################################################################
###################################################### CONFIG FILE ####################################################################
#######################################################################################################################################
sleep_time = 0 #minutes


config = {
    'function_family': {
        'maximum_depth': 4,
        'beta': 1,
        'decision_sparsity': 1,
        'fully_grown': True,                      
    },
    'data': {
        'number_of_variables': 66, 
        'num_classes': 2,
        
        'function_generation_type': 'make_classification_vanilla_decision_tree_trained', # 'make_classification' 'random_decision_tree' 'random_decision_tree_trained' 'random_vanilla_decision_tree_trained' 'make_classification_vanilla_decision_tree_trained'
        'objective': 'classification', # 'regression'
        
        'x_max': 1,
        'x_min': 0,
        'x_distrib': 'uniform', #'normal', 'uniform',       
                
        'lambda_dataset_size': 5000, #number of samples per function
        'number_of_generated_datasets': 10000,
    }, 
    'computation':{
        'n_jobs': -3,
        'use_gpu': False,
        'gpu_numbers': '0',
        'RANDOM_SEED': 0,   
    }
}


In [2]:
#######################################################################################################################################
########################################### IMPORT GLOBAL VARIABLES FROM CONFIG #######################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['computation'])

## Imports

In [3]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from itertools import product       # forms cartesian products
from more_itertools import random_product 
from tqdm import tqdm_notebook as tqdm
import pickle

import numpy as np

import pandas as pd
from joblib import Parallel, delayed

import random 
from random import sample 

import os
import sys

from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler

from sympy import Symbol, sympify

        
import seaborn as sns
        
import random 

import warnings

from time import perf_counter



from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from utilities.DecisionTree_BASIC import SDT
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree
from matplotlib import pyplot as plt
from IPython.display import Image

import numpy as np
import torch
np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

In [4]:
#######################################################################################################################################
###################################################### SET VARIABLES + DESIGN #########################################################
#######################################################################################################################################
sns.set_style("darkgrid")

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
    
    
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 500)

warnings.filterwarnings('ignore')

In [5]:
from utilities.utility_functions import *
#######################################################################################################################################
####################################################### CONFIG ADJUSTMENTS ############################################################
#######################################################################################################################################

#######################################################################################################################################
################################################## UPDATE VARIABLES ###################################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['computation'])

#initialize_utility_functions_config_from_curent_notebook(config)

#######################################################################################################################################
###################################################### PATH + FOLDER CREATION #########################################################
#######################################################################################################################################
globals().update(generate_paths(config, path_type='data_creation'))
generate_directory_structure()

#######################################################################################################################################
############################################################ SLEEP TIMER ##############################################################
#######################################################################################################################################
sleep_minutes(sleep_time)

In [6]:
print(path_identifier_function_data)

lNetSize5000_numDatasets10000_var66_class2_make_classification_vanilla_decision_tree_trained_xMax1_xMin0_xDistuniform_depth4_beta1_decisionSpars1_fullyGrown


# Function Generation

In [7]:
if function_generation_type == 'random_decision_tree':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_decision_tree_data)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))
elif function_generation_type == 'random_decision_tree_trained':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_decision_tree_data_trained)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))  
elif function_generation_type == 'make_classification':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_decision_tree_data_trained_make_classification)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets)) 
elif function_generation_type == 'make_classification_vanilla_decision_tree_trained':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_decision_tree_data_trained_make_classification_vanilla_decision_tree_trained)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))    
elif function_generation_type == 'random_vanilla_decision_tree_trained':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_decision_tree_data_random_vanilla_decision_tree_trained)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets)) 

function_identifier_list = generate_decision_tree_identifier(config)  
identifier_series_list = [pd.Series(result[0],  index=function_identifier_list) for result in result_list]

function_df = pd.DataFrame(data=np.array([result[0] for result in result_list]), columns=function_identifier_list)

X_data_list = [[identifier_series, pd.DataFrame(result[1], columns=['x' + str(i) for i in range(number_of_variables)])] for identifier_series, result in zip(identifier_series_list, result_list)]
y_data_list = [[identifier_series, pd.DataFrame(result[2], columns=['result'])] for identifier_series, result in zip(identifier_series_list, result_list)]
y_data_raw_list = [[identifier_series, pd.DataFrame(result[3], columns=['result_raw'])] for identifier_series, result in zip(identifier_series_list, result_list)]


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-3)]: Done 1108 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-3)]: Done 1524 tasks      | elapsed:   28.8s
[Parallel(n_jobs=-3)]: Done 2004 tasks      | elapsed:   35.0s
[Parallel(n_jobs=-3)]: Done 2548 tasks      | elapsed:   41.9s
[Parallel(n_jobs=-3)]: Done 3156 tasks      | elapsed:   50.0s
[Parallel(n_jobs=-3)]: Done 3828 tasks      | elapsed:   58.7s
[Parallel(n_jobs=-3)]: Done 4564 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-3)]: Done 5364 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-3)]: Done 6228 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-3)]: Done 7156 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-3)]: Done 8148 tasks      |

In [8]:
function_df.head()

,f0v0,f0v1,f0v2,f0v3,f0v4,f0v5,f0v6,f0v7,f0v8,f0v9,f0v10,f0v11,f0v12,f0v13,f0v14,f0v15,f0v16,f0v17,f0v18,f0v19,f0v20,f0v21,f0v22,f0v23,f0v24,f0v25,f0v26,f0v27,f0v28,f0v29,f0v30,f0v31,f0v32,f0v33,f0v34,f0v35,f0v36,f0v37,f0v38,f0v39,f0v40,f0v41,f0v42,f0v43,f0v44,f0v45,f0v46,f0v47,f0v48,f0v49,f0v50,f0v51,f0v52,f0v53,f0v54,f0v55,f0v56,f0v57,f0v58,f0v59,f0v60,f0v61,f0v62,f0v63,f0v64,f0v65,f1v0,f1v1,f1v2,f1v3,f1v4,f1v5,f1v6,f1v7,f1v8,f1v9,f1v10,f1v11,f1v12,f1v13,f1v14,f1v15,f1v16,f1v17,f1v18,f1v19,f1v20,f1v21,f1v22,f1v23,f1v24,f1v25,f1v26,f1v27,f1v28,f1v29,f1v30,f1v31,f1v32,f1v33,f1v34,f1v35,f1v36,f1v37,f1v38,f1v39,f1v40,f1v41,f1v42,f1v43,f1v44,f1v45,f1v46,f1v47,f1v48,f1v49,f1v50,f1v51,f1v52,f1v53,f1v54,f1v55,f1v56,f1v57,f1v58,f1v59,f1v60,f1v61,f1v62,f1v63,f1v64,f1v65,f2v0,f2v1,f2v2,f2v3,f2v4,f2v5,f2v6,f2v7,f2v8,f2v9,f2v10,f2v11,f2v12,f2v13,f2v14,f2v15,f2v16,f2v17,f2v18,f2v19,f2v20,f2v21,f2v22,f2v23,f2v24,f2v25,f2v26,f2v27,f2v28,f2v29,f2v30,f2v31,f2v32,f2v33,f2v34,f2v35,f2v36,f2v37,f2v38,f2v39,f2v40,f2v41,f2v42,f2v43,f2v44,f2v45,f2v46,f2v47,f2v48,f2v49,f2v50,f2v51,f2v52,f2v53,f2v54,f2v55,f2v56,f2v57,f2v58,f2v59,f2v60,f2v61,f2v62,f2v63,f2v64,f2v65,f3v0,f3v1,f3v2,f3v3,f3v4,f3v5,f3v6,f3v7,f3v8,f3v9,f3v10,f3v11,f3v12,f3v13,f3v14,f3v15,f3v16,f3v17,f3v18,f3v19,f3v20,f3v21,f3v22,f3v23,f3v24,f3v25,f3v26,f3v27,f3v28,f3v29,f3v30,f3v31,f3v32,f3v33,f3v34,f3v35,f3v36,f3v37,f3v38,f3v39,f3v40,f3v41,f3v42,f3v43,f3v44,f3v45,f3v46,f3v47,f3v48,f3v49,f3v50,f3v51,...,f11v61,f11v62,f11v63,f11v64,f11v65,f12v0,f12v1,f12v2,f12v3,f12v4,f12v5,f12v6,f12v7,f12v8,f12v9,f12v10,f12v11,f12v12,f12v13,f12v14,f12v15,f12v16,f12v17,f12v18,f12v19,f12v20,f12v21,f12v22,f12v23,f12v24,f12v25,f12v26,f12v27,f12v28,f12v29,f12v30,f12v31,f12v32,f12v33,f12v34,f12v35,f12v36,f12v37,f12v38,f12v39,f12v40,f12v41,f12v42,f12v43,f12v44,f12v45,f12v46,f12v47,f12v48,f12v49,f12v50,f12v51,f12v52,f12v53,f12v54,f12v55,f12v56,f12v57,f12v58,f12v59,f12v60,f12v61,f12v62,f12v63,f12v64,f12v65,f13v0,f13v1,f13v2,f13v3,f13v4,f13v5,f13v6,f13v7,f13v8,f13v9,f13v10,f13v11,f13v12,f13v13,f13v14,f13v15,f13v16,f13v17,f13v18,f13v19,f13v20,f13v21,f13v22,f13v23,f13v24,f13v25,f13v26,f13v27,f13v28,f13v29,f13v30,f13v31,f13v32,f13v33,f13v34,f13v35,f13v36,f13v37,f13v38,f13v39,f13v40,f13v41,f13v42,f13v43,f13v44,f13v45,f13v46,f13v47,f13v48,f13v49,f13v50,f13v51,f13v52,f13v53,f13v54,f13v55,f13v56,f13v57,f13v58,f13v59,f13v60,f13v61,f13v62,f13v63,f13v64,f13v65,f14v0,f14v1,f14v2,f14v3,f14v4,f14v5,f14v6,f14v7,f14v8,f14v9,f14v10,f14v11,f14v12,f14v13,f14v14,f14v15,f14v16,f14v17,f14v18,f14v19,f14v20,f14v21,f14v22,f14v23,f14v24,f14v25,f14v26,f14v27,f14v28,f14v29,f14v30,f14v31,f14v32,f14v33,f14v34,f14v35,f14v36,f14v37,f14v38,f14v39,f14v40,f14v41,f14v42,f14v43,f14v44,f14v45,f14v46,f14v47,f14v48,f14v49,f14v50,f14v51,f14v52,f14v53,f14v54,f14v55,f14v56,f14v57,f14v58,f14v59,f14v60,f14v61,f14v62,f14v63,f14v64,f14v65,b0,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,b11,b12,b13,b14,lp0c0,lp0c1,lp1c0,lp1c1,lp2c0,lp2c1,lp3c0,lp3c1,lp4c0,lp4c1,lp5c0,lp5c1,lp6c0,lp6c1,lp7c0,lp7c1,lp8c0,lp8c1,lp9c0,lp9c1,lp10c0,lp10c1,lp11c0,lp11c1,lp12c0,lp12c1,lp13c0,lp13c1,lp14c0,lp14c1,lp15c0,lp15c1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [9]:
X_data_list[0][0].head(10)

f0v0    0
f0v1    0
f0v2    0
f0v3    0
f0v4    0
f0v5    0
f0v6    0
f0v7    0
f0v8    0
f0v9    0
dtype: int64

In [10]:
X_data_list[0][1].head(10)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65
0,0.656,0.553,0.613,0.461,0.358,0.580,0.802,0.500,0.517,0.447,0.509,0.542,0.539,0.596,0.364,0.369,0.432,0.692,0.526,0.504,0.686,0.636,0.369,0.389,0.489,0.480,0.395,0.660,0.503,0.516,0.575,0.547,0.524,0.430,0.567,0.650,0.292,0.553,0.534,0.438,0.726,0.444,0.529,0.564,0.587,0.571,0.491,0.712,0.573,0.494,0.577,0.430,0.628,0.503,0.602,0.403,0.273,0.513,0.385,0.512,0.541,0.461,0.234,0.782,0.466,0.626
1,0.540,0.177,0.623,0.585,0.442,0.610,0.313,0.288,0.367,0.385,0.448,0.396,0.417,0.434,0.220,0.530,0.433,0.517,0.597,0.441,0.609,0.502,0.387,0.416,0.545,0.396,0.476,0.626,0.667,0.274,0.192,0.386,0.451,0.539,0.428,0.209,0.345,0.433,0.474,0.514,0.566,0.680,0.377,0.391,0.577,0.524,0.415,0.376,0.647,0.240,0.440,0.553,0.771,0.609,0.423,0.759,0.446,0.749,0.502,0.614,0.482,0.482,0.665,0.651,0.579,0.470
2,0.565,0.539,0.444,0.687,0.708,0.276,0.328,0.544,0.497,0.321,0.405,0.471,0.536,0.449,0.423,0.465,0.404,0.590,0.523,0.296,0.469,0.366,0.655,0.584,0.307,0.421,0.414,0.518,0.532,0.208,0.518,0.458,0.705,0.657,0.455,0.512,0.576,0.546,0.413,0.544,0.411,0.388,0.494,0.405,0.748,0.456,0.478,0.319,0.519,0.641,0.602,0.454,0.689,0.433,0.395,0.320,0.505,0.453,0.562,0.657,0.571,0.593,0.709,0.368,0.393,0.508
3,0.592,0.412,0.512,0.520,0.607,0.643,0.619,0.408,0.459,0.611,0.413,0.457,0.435,0.888,0.475,0.364,0.343,0.503,0.565,0.503,0.231,0.728,0.421,0.617,0.419,0.351,0.641,0.437,0.338,0.713,0.706,0.512,0.543,0.684,0.340,0.755,0.356,0.285,0.398,0.498,0.603,0.480,0.514,0.320,0.544,0.339,0.635,0.521,0.490,0.361,0.376,0.354,0.633,0.527,0.469,0.433,0.581,0.409,0.421,0.630,0.378,0.536,0.524,0.584,0.434,0.460
4,0.723,0.465,0.450,0.573,0.287,0.327,0.589,0.611,0.504,0.566,0.626,0.242,0.490,0.381,0.491,0.526,0.370,0.625,0.410,0.792,0.550,0.530,0.527,0.426,0.551,0.195,0.394,0.708,0.383,0.662,0.372,0.571,0.596,0.540,0.831,0.694,0.464,0.506,0.536,0.761,0.539,0.336,0.648,0.586,0.607,0.659,0.587,0.553,0.592,0.559,0.406,0.492,0.441,0.618,0.472,0.268,0.685,0.432,0.539,0.406,0.506,0.449,0.444,0.488,0.417,0.805
5,0.529,0.583,0.352,0.578,0.306,0.624,0.595,0.397,0.422,0.238,0.550,0.493,0.458,0.612,0.607,0.626,0.245,0.244,0.639,0.384,0.299,0.652,0.251,0.241,0.456,0.643,0.514,0.502,0.416,0.910,0.470,0.396,0.506,0.596,0.283,0.494,0.617,0.465,0.544,0.490,0.441,0.471,0.508,0.267,0.594,0.531,0.365,0.797,0.334,0.385,0.448,0.583,0.597,0.683,0.514,0.483,0.704,0.451,0.490,0.597,0.686,0.686,0.265,0.552,0.580,0.486
6,0.754,0.457,0.544,0.444,0.508,0.487,0.507,0.523,0.434,0.517,0.706,0.553,0.522,0.708,0.472,0.568,0.483,0.376,0.499,0.804,0.398,0.675,0.598,0.555,0.635,0.475,0.396,0.742,0.622,0.345,0.673,0.600,0.666,0.532,0.288,0.432,0.490,0.578,0.402,0.535,0.492,0.341,0.594,0.375,0.516,0.452,0.601,0.226,0.488,0.836,0.433,0.319,0.369,0.488,0.554,0.334,0.478,0.551,0.652,0.317,0.426,0.446,0.454,0.540,0.491,0.272
7,0.522,0.375,0.484,0.599,0.426,0.282,0.749,0.337,0.389,0.478,0.478,0.622,0.457,0.596,0.509,0.375,0.522,0.531,0.627,0.367,0.333,0.725,0.585,0.398,0.405,0.446,0.506,0.540,0.415,0.267,0.490,0.535,0.636,0.605,0.575,0.559,0.432,0.727,0.339,0.628,0.475,0.615,0.475,0.616,0.557,0.702,0.770,0.477,0.386,0.313,0.592,0.634,0.506,0.460,0.559,0.270,0.522,0.703,0.370,0.393,0.506,0.605,0.323,0.646,0.564,0.594
8,0.645,0.481,0.496,0.574,0.481,0.344,0.354,0.673,0.557,0.415,0.446,0.333,0.616,0.558,0.533,0.646,0.378,0.740,0.441,0.678,0.622,0.403,0.298,0.698,0.244,0.473,0.568,0.616,0.202,0.428,0.447,0.516,0.532,0.522,0.586,0.253,0.702,0.618,0.590,0.497,0.610,0.537,0.711,0.612,0.586,0.403,0.803,0.415,0.649,0.413,0.598,0.335,0.459,0.357,0.416,0.452,0.506,0.533,0.451,0.436,0.510,0.581,0.393,0.430,0.372,0.558
9,0.482,0.625,0.467,0.474,0.549,0.474,0.636,0.588,0.578,0.557,0.450,0.382,0.628,0.461,0.516,0.535,0.361,0.462,0.568,0.677,0.434,0.691,0.665,0.399,0.602,0.389,0.516

In [11]:
y_data_list[0][0].head(10)

f0v0    0
f0v1    0
f0v2    0
f0v3    0
f0v4    0
f0v5    0
f0v6    0
f0v7    0
f0v8    0
f0v9    0
dtype: int64

In [12]:
y_data_list[2][1].head(10)

,result
0,0
1,1
2,1
3,1
4,0
5,0
6,0
7,1
8,1
9,0


In [13]:
y_data_raw_list[2][1].head(10)

,result_raw
0,0
1,1
2,1
3,1
4,0
5,0
6,0
7,1
8,1
9,0


In [14]:
#some_tree = generate_decision_tree_from_array(y_data_list[0][0].values, config)

In [15]:
#some_tree.plot_tree()


In [16]:
path_functions = './data/saved_function_lists/functions_' + path_identifier_function_data + '.csv'
function_df.to_csv(path_functions, index=False)

path_X_data = './data/saved_function_lists/X_data_' + path_identifier_function_data + '.pkl'
with open(path_X_data, 'wb') as f:
    pickle.dump(X_data_list, f)
    
path_y_data = './data/saved_function_lists/y_data_' + path_identifier_function_data + '.pkl'
with open(path_y_data, 'wb') as f:
    pickle.dump(y_data_list, f)
